[Pytorch](https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html) tutorial

In [60]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.autograd import Variable as V
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import time
import math
import csv
import gzip

In [54]:
# Utility methods

def str2ascii_arr(msg):
    arr = [ord(c) for c in msg]
    return arr, len(arr)

def pad_sequences(vectorized_seqs, seq_lengths, countries):
    seq_tensor = torch.zeros((len(vectorized_seqs), seq_lengths.max())).long()
    for idx, (seq, seq_len) in enumerate(zip(vectorized_seqs, seq_lengths)):
        seq_tensor[idx, :seq_len] = torch.LongTensor(seq)

    # Sort tensors by their length
    seq_lengths, perm_idx = seq_lengths.sort(0, descending=True)
    seq_tensor = seq_tensor[perm_idx]

    # Also sort the target (countries) in the same order
    target = countries2tensor(countries)
    if len(countries):
        target = target[perm_idx]

    # Return variables
    # DataParallel requires everything to be a Variable
    return V(seq_tensor), V(seq_lengths), V(target)

# def pad_sequences(vectorized_seqs, seq_lengths):
#     seq_tensor = torch.zeros((len(vectorized_seqs), seq_lengths.max())).long()
    
#     for idx, (seq, seq_length) in enumerate(zip(vectorized_seqs, seq_lengths)):
#         seq_tensor[idx, :seq_length] = torch.LongTensor(seq)
    
#     return seq_tensor

def make_variables(names, countries):
    sequence_and_length = [str2ascii_arr(name) for name in names]
    vectorized_seqs = [sl[0] for sl in sequence_and_length]
    seq_lengths = torch.LongTensor([sl[1] for sl in sequence_and_length])
    return pad_sequences(vectorized_seqs, seq_lengths, countries)

# def make_variables(names):
#     sequence_and_length = [str2ascii_arr(name) for name in names]
#     vectorized_seqs = [s1[0] for s1 in sequence_and_length]
#     seq_lengths = torch.LongTensor([s1[1] for s1 in sequence_and_length])
#     return pad_sequences(vectorized_seqs, seq_lengths)

def countries2tensor(countries):
    country_ids = [train_dataset.get_country_id(
        country) for country in countries]
    return torch.LongTensor(country_ids)


def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [45]:
# Parameters and DataLoaders
HIDDEN_SIZE = 100
N_LAYERS = 2
BATCH_SIZE = 256
N_CHARS = 128 # number of ASCII chars

class NameDataset(Dataset):
    # Initialize your data, download, etc.
    def __init__(self, is_train_set=False):
        filename = './data/names_train.csv.gz' if is_train_set else './data/names_test.csv.gz'
        with gzip.open(filename, "rt") as f:
            reader = csv.reader(f)
            rows = list(reader)

        self.names = [row[0] for row in rows]
        self.countries = [row[1] for row in rows]
        self.len = len(self.countries)

        self.country_list = list(sorted(set(self.countries)))

    def __getitem__(self, index):
        return self.names[index], self.countries[index]

    def __len__(self):
        return self.len

    def get_countries(self):
        return self.country_list

    def get_country(self, id):
        return self.country_list[id]

    def get_country_id(self, country):
        return self.country_list.index(country)

In [50]:
class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNNClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.fc = nn.Linear(hidden_size, output_size)
#         super(RNNClassifier, self).__init__()
#         self.hidden_size = hidden_size
#         self.n_layers = n_layers
#         self.n_directions = int(bidirectional) + 1

#         self.embedding = nn.Embedding(input_size, hidden_size)
#         self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
#                           bidirectional=bidirectional)
#         self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, seq_lengths):
        input = input.t()
        batch_size = input.size(1)

        # Make a hidden
        hidden = self._init_hidden(batch_size)

        # Embedding S x B -> S x B x I (embedding size)
        embedded = self.embedding(input)

        # Pack them up nicely
        gru_input = pack_padded_sequence(
            embedded, seq_lengths.data.cpu().numpy())

        # To compact weights again call flatten_parameters().
        self.gru.flatten_parameters()
        output, hidden = self.gru(gru_input, hidden)

        # Use the last layer output as FC's input
        # No need to unpack, since we are going to use hidden
        fc_output = self.fc(hidden[-1])
        return fc_output
#         batch_size = input.size(0)
#         input = input.t()
#         print("input", input.size())
#         embedded = self.embedding(input)
#         print("embedding", embedded)
        
#         hidden = self._init_hidden(batch_size)
#         output, hidden = self.gru(embedded, hidden)
#         print("gru hidden output", hidden.size())
        
#         fc_output = self.fc(hidden)
#         print("fc output", fc_output.size())
#         return fc_output
    
    def _init_hidden(self, batch_size):
#         hidden = torch.zeros(self.n_layers * self.n_directions,
#         batch_size, self.hidden_size)
#         return create_variable(hidden)
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_size)
        return V(hidden)

In [51]:
classifier = RNNClassifier(N_CHARS, HIDDEN_SIZE, N_COUNTRIES, N_LAYERS)

optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

test_dataset = NameDataset(is_train_set=False)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=BATCH_SIZE, 
                         shuffle=True)


train_dataset = NameDataset(is_train_set=True)
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE, 
                          shuffle=True)

N_COUNTRIES = len(train_dataset.get_countries())
print(N_COUNTRIES, "countries")

18 countries


In [58]:
def train():
    total_loss = 0
    start = time.time()
    
    for i, (names, countries) in enumerate(train_loader, 1):
        input, seq_lengths, target = make_variables(names, countries)
        output = classifier(input, seq_lengths)
        loss = criterion(output, target)
        total_loss += loss.data[0]

        classifier.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print('[{}] Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.2f}'.format(
                time_since(start), epoch,  i *
                len(names), len(train_loader.dataset),
                100. * i * len(names) / len(train_loader.dataset),
                total_loss / i * len(names)))

    return total_loss

def test(name=None):
    # Predict for a given name
    if name:
        input, seq_lengths, target = make_variables([name], [])
        output = classifier(input, seq_lengths)
        pred = output.data.max(1, keepdim=True)[1]
        country_id = pred.cpu().numpy()[0][0]
        print(name, "is", train_dataset.get_country(country_id))
        return

    print("evaluating trained model ...")
    correct = 0
    train_data_size = len(test_loader.dataset)

    for names, countries in test_loader:
        input, seq_lengths, target = make_variables(names, countries)
        output = classifier(input, seq_lengths)
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, train_data_size, 100. * correct / train_data_size))

In [62]:
for epoch in range(1, 101):
    # Train cycle
    train()

    # Testing
    test()

    # Testing several samples
    test("Sung")
    test("Jungwoo")
    test("Soojin")
    test("Nako")

 0s] Train Epoch: 1 [2560/13374 (19%)]	Loss: 275.05
 1s] Train Epoch: 1 [5120/13374 (38%)]	Loss: 271.21
 1s] Train Epoch: 1 [7680/13374 (57%)]	Loss: 268.12
 2s] Train Epoch: 1 [10240/13374 (77%)]	Loss: 262.21
 3s] Train Epoch: 1 [12800/13374 (96%)]	Loss: 259.19
Sung is Chinese
Jungwoo is Italian
Soojin is Russian
Nako is Japanese
 0s] Train Epoch: 2 [2560/13374 (19%)]	Loss: 225.64
 1s] Train Epoch: 2 [5120/13374 (38%)]	Loss: 224.75
 1s] Train Epoch: 2 [7680/13374 (57%)]	Loss: 220.82
 2s] Train Epoch: 2 [10240/13374 (77%)]	Loss: 218.57
 3s] Train Epoch: 2 [12800/13374 (96%)]	Loss: 216.94
Sung is Chinese
Jungwoo is Italian
Soojin is Russian
Nako is Japanese
 0s] Train Epoch: 3 [2560/13374 (19%)]	Loss: 200.56
 1s] Train Epoch: 3 [5120/13374 (38%)]	Loss: 192.86
 1s] Train Epoch: 3 [7680/13374 (57%)]	Loss: 192.74
 2s] Train Epoch: 3 [10240/13374 (77%)]	Loss: 191.42
 3s] Train Epoch: 3 [12800/13374 (96%)]	Loss: 190.00
Sung is Chinese
Jungwoo is Italian
Soojin is Russian
Nako is Japanese
 0s]

 0s] Train Epoch: 25 [2560/13374 (19%)]	Loss: 31.39
 1s] Train Epoch: 25 [5120/13374 (38%)]	Loss: 31.00
 2s] Train Epoch: 25 [7680/13374 (57%)]	Loss: 30.61
 2s] Train Epoch: 25 [10240/13374 (77%)]	Loss: 30.95
 3s] Train Epoch: 25 [12800/13374 (96%)]	Loss: 33.41
Sung is Chinese
Jungwoo is Korean
Soojin is Polish
Nako is Japanese
 0s] Train Epoch: 26 [2560/13374 (19%)]	Loss: 31.80
 1s] Train Epoch: 26 [5120/13374 (38%)]	Loss: 30.00
 1s] Train Epoch: 26 [7680/13374 (57%)]	Loss: 30.35
 2s] Train Epoch: 26 [10240/13374 (77%)]	Loss: 30.63
 3s] Train Epoch: 26 [12800/13374 (96%)]	Loss: 31.03
Sung is Chinese
Jungwoo is Korean
Soojin is German
Nako is Japanese
 0s] Train Epoch: 27 [2560/13374 (19%)]	Loss: 25.64
 1s] Train Epoch: 27 [5120/13374 (38%)]	Loss: 25.81
 1s] Train Epoch: 27 [7680/13374 (57%)]	Loss: 27.88
 2s] Train Epoch: 27 [10240/13374 (77%)]	Loss: 28.73
 3s] Train Epoch: 27 [12800/13374 (96%)]	Loss: 29.17
Sung is Chinese
Jungwoo is Korean
Soojin is Polish
Nako is Japanese
 0s] Train

 0s] Train Epoch: 49 [2560/13374 (19%)]	Loss: 11.15
 1s] Train Epoch: 49 [5120/13374 (38%)]	Loss: 11.93
 1s] Train Epoch: 49 [7680/13374 (57%)]	Loss: 12.57
 2s] Train Epoch: 49 [10240/13374 (77%)]	Loss: 12.81
 2s] Train Epoch: 49 [12800/13374 (96%)]	Loss: 12.74
Sung is Chinese
Jungwoo is Korean
Soojin is German
Nako is Japanese
 0s] Train Epoch: 50 [2560/13374 (19%)]	Loss: 12.45
 1s] Train Epoch: 50 [5120/13374 (38%)]	Loss: 12.41
 1s] Train Epoch: 50 [7680/13374 (57%)]	Loss: 12.59
 2s] Train Epoch: 50 [10240/13374 (77%)]	Loss: 13.59
 2s] Train Epoch: 50 [12800/13374 (96%)]	Loss: 13.62
Sung is Chinese
Jungwoo is Korean
Soojin is English
Nako is Japanese
 0s] Train Epoch: 51 [2560/13374 (19%)]	Loss: 10.98
 1s] Train Epoch: 51 [5120/13374 (38%)]	Loss: 13.04
 1s] Train Epoch: 51 [7680/13374 (57%)]	Loss: 12.38
 2s] Train Epoch: 51 [10240/13374 (77%)]	Loss: 12.86
 3s] Train Epoch: 51 [12800/13374 (96%)]	Loss: 13.13
Sung is Chinese
Jungwoo is Korean
Soojin is English
Nako is Japanese
 0s] Tra

 0s] Train Epoch: 73 [2560/13374 (19%)]	Loss: 9.10
 1s] Train Epoch: 73 [5120/13374 (38%)]	Loss: 9.55
 1s] Train Epoch: 73 [7680/13374 (57%)]	Loss: 10.39
 2s] Train Epoch: 73 [10240/13374 (77%)]	Loss: 10.59
 2s] Train Epoch: 73 [12800/13374 (96%)]	Loss: 10.95
Sung is Chinese
Jungwoo is Korean
Soojin is English
Nako is Japanese
 0s] Train Epoch: 74 [2560/13374 (19%)]	Loss: 9.12
 1s] Train Epoch: 74 [5120/13374 (38%)]	Loss: 8.52
 1s] Train Epoch: 74 [7680/13374 (57%)]	Loss: 9.28
 2s] Train Epoch: 74 [10240/13374 (77%)]	Loss: 10.20
 2s] Train Epoch: 74 [12800/13374 (96%)]	Loss: 10.72
Sung is Chinese
Jungwoo is Korean
Soojin is English
Nako is Japanese
 0s] Train Epoch: 75 [2560/13374 (19%)]	Loss: 9.31
 1s] Train Epoch: 75 [5120/13374 (38%)]	Loss: 9.94
 1s] Train Epoch: 75 [7680/13374 (57%)]	Loss: 10.15
 2s] Train Epoch: 75 [10240/13374 (77%)]	Loss: 9.88
 2s] Train Epoch: 75 [12800/13374 (96%)]	Loss: 10.32
Sung is Chinese
Jungwoo is Korean
Soojin is English
Nako is Japanese
 0s] Train Epoc

 0s] Train Epoch: 97 [2560/13374 (19%)]	Loss: 8.30
 1s] Train Epoch: 97 [5120/13374 (38%)]	Loss: 8.41
 1s] Train Epoch: 97 [7680/13374 (57%)]	Loss: 8.84
 2s] Train Epoch: 97 [10240/13374 (77%)]	Loss: 9.28
 3s] Train Epoch: 97 [12800/13374 (96%)]	Loss: 9.60
Sung is Chinese
Jungwoo is Korean
Soojin is English
Nako is Japanese
 0s] Train Epoch: 98 [2560/13374 (19%)]	Loss: 8.42
 1s] Train Epoch: 98 [5120/13374 (38%)]	Loss: 8.50
 1s] Train Epoch: 98 [7680/13374 (57%)]	Loss: 9.18
 2s] Train Epoch: 98 [10240/13374 (77%)]	Loss: 9.48
 3s] Train Epoch: 98 [12800/13374 (96%)]	Loss: 9.76
Sung is Chinese
Jungwoo is Korean
Soojin is English
Nako is Japanese
 0s] Train Epoch: 99 [2560/13374 (19%)]	Loss: 7.12
 1s] Train Epoch: 99 [5120/13374 (38%)]	Loss: 7.73
 1s] Train Epoch: 99 [7680/13374 (57%)]	Loss: 8.78
 2s] Train Epoch: 99 [10240/13374 (77%)]	Loss: 9.23
 3s] Train Epoch: 99 [12800/13374 (96%)]	Loss: 9.59
Sung is Chinese
Jungwoo is Chinese
Soojin is English
Nako is Japanese
 0s] Train Epoch: 100

In [63]:
test("iacutone")

iacutone is Italian
